In [ ]:
import os
import pandas as pd
import pygwalker as pyg


import time

## 数据读取

In [ ]:
# 指定存储所有视频元数据的文件夹路径
metadata_folder = "Dataset/video_metadata"

all_video_metadata_dict = {}

current_video_metadata_dict = {}

sub_atribs = ["General", "Video", "Audio"]

current_sub_attrib = None

In [ ]:
# 遍历元数据文件夹中的所有文件
for file_name in os.listdir(metadata_folder):
    # 仅处理txt文件
    if file_name.endswith(".txt"):
        file_path = os.path.join(metadata_folder, file_name)
        video_name = file_name[:-4]

        with open(f"{file_path}", "r") as f:
            text = f.readlines()

            current_video_metadata_dict = {}

            for line in text:
                line = line.strip()
                if line in sub_atribs:
                    # 当读取到属性为预定义的子属性时，设置当前正在读取的子属性
                    current_sub_attrib = line

                elif current_sub_attrib and line:  # 非空字符串视为True
                    key, value = line.split(":", 1)
                    new_key = f"{current_sub_attrib}_{key.strip()}"    # 不同子类中有时会有重复名称的属性，为了避免重复，将属性名加上子属性名前缀
                    current_video_metadata_dict[new_key] = value.strip()

                elif current_sub_attrib and not line:  
                    # 如果当前行为空，则将当前子属性置为None，表示结束当前子属性的读取
                    current_sub_attrib = None

            all_video_metadata_dict[f"{video_name}"] = current_video_metadata_dict
            print(f"Finished reading {video_name}")
print("Finished reading all video metadata")

In [ ]:
# 将元数据列表转换为DataFrame对象
df = pd.DataFrame(all_video_metadata_dict)
df.head()

In [ ]:
# print(df.shape)
# print(df.iloc[4,0])

In [ ]:
# 取帧数敏感参数分析
para_list = ["General_Duration",
             "General_Frame rate",
             "Video_Duration",
             "Video_Source duration",
             "Video_Width",
             "Video_Height",
             "Video_Frame rate mode",
             "Video_Frame rate"]

fps_df = df.loc[para_list, :]
print(fps_df.shape)
fps_df = fps_df.transpose()

In [ ]:
walker = pyg.walk(fps_df)

## 标签数据处理

In [ ]:
clip_fps_df = fps_df.loc[:, ["Video_Duration",
             "Video_Source duration",
             "Video_Width",
             "Video_Height",
             "Video_Frame rate mode",
             "Video_Frame rate"]]

In [ ]:
# 将时间格式转换为数值
# clip_fps_df["Video_Duration"] = int(1)

clip_walker = pyg.walk(clip_fps_df)

In [ ]:
dt = '1min0s'

dt = time.strptime(dt, '%Mmin%Ss')
print(dt)

In [ ]:
clip_fps_df.loc[:,["Video_Frame rate","Video_Frame rate mode"]]

In [ ]:
clip_fps_df.to_csv("video_metadata.csv", index=True)

## 读取csv对照

In [ ]:
df1 = pd.read_csv("video_metadata.csv", index_col=0)
df2 = pd.read_csv("clip_data.csv", index_col=0)

In [ ]:
df2

In [ ]:
df3 = pd.concat([df1, df2], axis=1)
df3

In [ ]:
df3.loc[:,["clip count all"]]